In [67]:
"""INSTALLATION DE PANDAS ET DE PROPHET"""
!pip install pandas
!pip install prophet

In [68]:
import pandas as pd
dataset_path = 'jour_2024-01-09_au_2024-02-26.csv'
energy_dataset= pd.read_csv(dataset_path, sep=';',  encoding='latin1')
energy_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Contrat                   49 non-null     int64 
 1   Tarif                     49 non-null     object
 2   Date                      49 non-null     object
 3   kWh                       49 non-null     object
 4   Code de consommation      49 non-null     object
 5   Température moyenne (°C)  49 non-null     int64 
 6   Code de température       49 non-null     object
dtypes: int64(2), object(5)
memory usage: 2.8+ KB


In [69]:
energy_dataset.head()

,Contrat,Tarif,Date,kWh,Code de consommation,Température moyenne (°C),Code de température
0,316198211,D,2024-02-26,"26,05",R,2,R
1,316198211,D,2024-02-25,"32,04",R,-4,R
2,316198211,D,2024-02-24,"33,65",R,-12,R
3,316198211,D,2024-02-23,"24,39",R,1,R
4,316198211,D,2024-02-22,"25,56",R,2,R


In [70]:
#Supprimer les lignes contenant NaN
energy_dataset = energy_dataset.dropna(axis=0, how='any')

In [71]:
energy_dataset = energy_dataset.drop(columns = ['Contrat', 'Tarif', 'Code de consommation', 'Température moyenne (°C)', 'Code de température'], axis=1)

In [72]:
energy_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    49 non-null     object
 1   kWh     49 non-null     object
dtypes: object(2)
memory usage: 916.0+ bytes


In [73]:
energy_dataset['Date'] = pd.to_datetime(energy_dataset['Date'])
energy_dataset['kWh'] = energy_dataset['kWh'].str.replace(',', '.').astype('float64')

In [74]:
energy_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    49 non-null     datetime64[ns]
 1   kWh     49 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 916.0 bytes


In [75]:
energy_dataset.head()

,Date,kWh
0,2024-02-26,26.05
1,2024-02-25,32.04
2,2024-02-24,33.65
3,2024-02-23,24.39
4,2024-02-22,25.56


In [76]:
energy_dataset.tail()

,Date,kWh
44,2024-01-13,30.41
45,2024-01-12,32.27
46,2024-01-11,34.38
47,2024-01-10,27.86
48,2024-01-09,31.70


In [77]:
energy_dataset = energy_dataset.loc[::-1].reset_index(drop=True)
energy_dataset.tail()

,Date,kWh
44,2024-02-22,25.56
45,2024-02-23,24.39
46,2024-02-24,33.65
47,2024-02-25,32.04
48,2024-02-26,26.05


In [78]:
from prophet import Prophet

In [85]:
m = Prophet()
energy_dataset.columns = ['ds', 'y']
m.fit(energy_dataset.iloc[:40])

00:35:56 - cmdstanpy - INFO - Chain [1] start processing
00:35:56 - cmdstanpy - INFO - Chain [1] done processing


In [87]:
future_consumption_date = m.make_future_dataframe(periods=10)
future_consumption_date.tail(10)

,ds
40,2024-02-18
41,2024-02-19
42,2024-02-20
43,2024-02-21
44,2024-02-22
45,2024-02-23
46,2024-02-24
47,2024-02-25
48,2024-02-26
49,2024-02-27


In [88]:
forecast = m.predict(future_consumption_date)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(10)

,ds,yhat,yhat_lower,yhat_upper
40,2024-02-18,30.921682,24.495220,37.072397
41,2024-02-19,29.568747,22.896409,36.413277
42,2024-02-20,26.996999,19.983588,33.073704
43,2024-02-21,28.927083,22.722821,36.085641
44,2024-02-22,27.767817,21.263754,34.441821
45,2024-02-23,26.200085,19.614094,33.063638
46,2024-02-24,27.308172,20.567739,34.891867
47,2024-02-25,29.987653,23.037094,36.975431
48,2024-02-26,28.634718,22.162983,35.081760
49,2024-02-27,26.062969,19.782696,33.093504
